In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8368.81it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4675.07it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2766664.85it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8583.86it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4690.41it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 81310.86it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 4623.12it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4437.62it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 58514.43it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8450.58it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4565.38it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 14425.66it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [12]:
model = mz.models.DRMMTKS()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mask_value'] = -1
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMMTKS.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_3 (Dot)                     (None, 10, 100)      0           embedding[0

In [13]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [14]:
model.load_embedding_matrix(embedding_matrix)

In [15]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [16]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [ ]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 4s 44ms/step - loss: 0.8404

Validation: normalized_discounted_cumulative_gain@3(0): 0.5284643480792759 - normalized_discounted_cumulative_gain@5(0): 0.5855881746317924 - mean_average_precision(0): 0.548999143742844
Epoch 2/30
102/102 [==============================] - 3s 32ms/step - loss: 0.6019
Validation: normalized_discounted_cumulative_gain@3(0): 0.5844638671762555 - normalized_discounted_cumulative_gain@5(0): 0.6459823689857352 - mean_average_precision(0): 0.6020629920946378
Epoch 3/30
102/102 [==============================] - 3s 32ms/step - loss: 0.4248
Validation: normalized_discounted_cumulative_gain@3(0): 0.6075839941451231 - normalized_discounted_cumulative_gain@5(0): 0.6717387906783745 - mean_average_precision(0): 0.6243166584938736
Epoch 4/30
102/102 [==============================] - 3s 33ms/step - loss: 0.3250
Validation: normalized_discounted_cumulative_gain@3(0): 0.6104461923835603 - normalized_discounted_cumulativ